In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import pytz
import math

In [2]:
from metar import Metar


def bad_weather_classes(obs: str):
    metar = Metar.Metar(obs, strict=False)
    ceiling = _get_visibility_ceiling_coef(metar)
    wind = _get_wind_coef(metar)
    precip = _get_precipitation_coef(metar)
    freezing = _get_freezing_coef(metar)
    phenomena = _get_dangerous_phenomena_coef(metar)
    wspeed,wdir= _get_wind_speed_dir(metar)

    return (ceiling, wind, precip, freezing, phenomena,wdir,wspeed)


def _get_dangerous_phenomena_coef(metar: Metar.Metar):
    phenomena, showers = __dangerous_weather(metar.weather)
    cb, tcu, ts = __dangerous_clouds(metar.sky)
    if showers is not None and showers > 0:
        if cb == 12:
            ts = 18 if showers == 1 else 24

        if cb == 10 or tcu == 10:
            ts = 12 if showers == 1 else 20

        if cb == 6 or tcu == 8:
            ts = 10 if showers == 1 else 15

        if cb == 4 or tcu == 5:
            ts = 8 if showers == 1 else 12

        if tcu == 3:
            ts = 4 if showers == 1 else 6

    return max(i for i in [phenomena, cb, tcu, ts] if i is not None)


def __dangerous_weather(weather: []):
    phenomena = None
    showers = None
    for intensity, desc, precip, obs, other in weather:
        __phenomena = 0
        __showers = 0
        if other in ["FC", "DS", "SS"] or obs in ["VA", "SA"] or precip in ["GR", "PL"]:
            __phenomena = 24

        if desc == "TS":
            __phenomena = 30 if intensity == "+" else 24

        if precip == "GS":
            __phenomena = 18

        if phenomena is None or __phenomena > phenomena:
            phenomena = __phenomena

        if desc == "SH":
            __showers = 1 if intensity == "-" else 2

        if showers is None or __showers > showers:
            showers = __showers

    return (phenomena, showers)


def __dangerous_clouds(sky: []):
    cb = 0
    tcu = 0
    for cover, height, cloud in sky:
        __cb = 0
        __tcu = 0
        if cover == "OVC":
            if cloud == "TCU":
                __tcu = 10

            if cloud == "CB":
                __cb = 12

        if cover == "BKN":
            if cloud == "TCU":
                __tcu = 8

            if cloud == "CB":
                __cb = 10

        if cover == "SCT":
            if cloud == "TCU":
                __tcu = 5

            if cloud == "CB":
                __cb = 6

        if cover == "FEW":
            if cloud == "TCU":
                __tcu = 3

            if cloud == "CB":
                __cb = 4

        if __cb > cb:
            cb = __cb

        if __tcu > tcu:
            tcu = __tcu

    return (cb, tcu, None)


def _get_wind_coef(metar: Metar.Metar):
    spd = metar.wind_speed.value() if metar.wind_speed is not None else None
    gusts = metar.wind_gust.value() if metar.wind_gust is not None else None
    coef = 0

    if spd is not None:
        if 16 <= spd <= 20:
            coef = 1

        if 21 <= spd <= 30:
            coef = 2

        if spd > 30:
            coef = 4

        if gusts is not None:
            coef += 1

    return coef


def _get_wind_speed_dir(metar: Metar.Metar):
    wspd = metar.wind_speed.value()
    wdir = metar.wind_dir

    if not (wdir==None):
        wdir=wdir.value()
    else:
        wdir=0
    return wspd,wdir


def _get_precipitation_coef(metar: Metar.Metar):
    coef = 0
    for intensity, desc, precip, obs, other in metar.weather:
        __coef = 0
        if desc == "FZ":
            __coef = 3

        if precip == "SN":
            __coef = 2 if intensity == "-" else 3

        if precip == "SG" or (precip == "RA" and intensity == "+"):
            __coef = 2

        if precip in ["RA", "UP", "IC", "DZ"]:
            __coef = 1

        if __coef > coef:
            coef = __coef

    return coef


def _get_freezing_coef(metar: Metar.Metar):
    tt = metar.temp.value() if metar.temp is not None else None
    dp = metar.dewpt.value() if metar.dewpt is not None else None
    moisture = None
    for intensity, desc, precip, obs, other in metar.weather:
        __moisture = 0
        if desc == "FZ":
            __moisture = 5

        if precip == "SN":
            __moisture = 4 if intensity == "-" else 5

        if precip in ["SG", "RASN"] or (precip == "RA" and intensity == "+") or obs == "BR":
            __moisture = 4

        if precip in ["DZ", "IC", "RA", "UP", "GR", "GS", "PL"] or obs == "FG":
            __moisture = 3

        if moisture is None or __moisture > moisture:
            moisture = __moisture

    if tt is not None and tt <= 3 and moisture == 5:
        return 4

    if tt is not None and tt <= -15 and moisture is not None:
        return 4

    if tt is not None and tt <= 3 and moisture == 4:
        return 3

    if tt is not None and tt <= 3 and (moisture == 3 or (tt - dp) < 3):
        return 1

    if tt is not None and tt <= 3 and moisture is None:
        return 0

    if tt is not None and tt > 3 and moisture is not None:
        return 0

    if tt is not None and tt > 3 and (moisture is None or (tt - dp) >= 3):
        return 0

    return 0




def _get_visibility_ceiling_coef(metar: Metar.Metar):
    vis = __get_visibility(metar)
    is_covered, cld_base = __get_ceiling(metar)

    if vis is not None and cld_base is not None:
        if (vis <= 325) or (is_covered and cld_base <= 50):
            return 5

        if (350 <= vis <= 500) or (is_covered and 100 <= cld_base <= 150):
            return 4

        if (550 <= vis <= 750) or (is_covered and 200 <= cld_base <= 250):
            return 2

    return 0



def __get_ceiling(metar: Metar.Metar):
    cld_cover = False
    cld_base = None
    for cover, height, cloud in metar.sky:
        if cover in ["BKN", "OVC"]:
            cld_cover = True

        if height is not None and (cld_base is None or cld_base > height.value()):
            cld_base = height.value()

    return (cld_cover, cld_base)


def __get_visibility(metar: Metar.Metar):
    vis = None
    if metar.vis is not None:
        vis = metar.vis.value()
    rvr = None
    for name, low, high, unit in metar.runway:
        if rvr is None or rvr > low.value():
            rvr = low.value()

    if rvr is not None and rvr < 1500:
        vis = rvr

    return vis

In [3]:
"""
Example script that scrapes data from the IEM ASOS download service.
Requires: Python 3
"""
import datetime
import json
import os
import sys
import time
import pandas as pd
from urllib.request import urlopen

# Number of attempts to download data
MAX_ATTEMPTS = 6
# HTTPS here can be problematic for installs that don't have Lets Encrypt CA
SERVICE = "http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?"

df1= None
startts= None
endts= None
stations=None

def download_data(uri):
    """Fetch the data from the IEM
    The IEM download service has some protections in place to keep the number
    of inbound requests in check.  This function implements an exponential
    backoff to keep individual downloads from erroring.
    Args:
      uri (string): URL to fetch
    Returns:
      string data
    """
    attempt = 0
    while attempt < MAX_ATTEMPTS:
        try:
            data = urlopen(uri, timeout=300).read().decode("utf-8")
            if data is not None and not data.startswith("ERROR"):
                return data
        except Exception as exp:
            print(f"download_data({uri}) failed with {exp}")
            time.sleep(5)
        attempt += 1

    print("Exhausted attempts to download, returning empty data")
    return ""


def main_weather(startts,endts,stations):
    """Our main method"""
    #global df1 , endts, startts , stations
    # timestamps in UTC to request data for
    #startts = datetime.datetime(2021, 12, 1)
    #endts = datetime.datetime(2022, 2, 1)

    service = SERVICE + "data=all&tz=Etc/UTC&format=comma&latlon=yes&"

    service += startts.strftime("year1=%Y&month1=%m&day1=%d&")
    service += endts.strftime("year2=%Y&month2=%m&day2=%d&")

    # Specify the airport station code
    #stations = ["EGLL"]
    for station in stations:
        uri = f"{service}&station={station}"
        print(f"Downloading: {station}")
        data = download_data(uri)
        outfn = f"{station}_{startts:%Y%m%d%H%M}_{endts:%Y%m%d%H%M}.txt"
        with open(outfn, "w", encoding="ascii") as fh:
            fh.write(data)
        df1 = pd.read_csv(f"C:\\Users\mic__\{station}_{startts:%Y%m%d%H%M}_{endts:%Y%m%d%H%M}.txt", sep = ',', header=5)


    df2=df1[["metar","valid"]]
    df2

    final = pd.DataFrame(columns=['metar', 'valid', 'ceiling', 'wind', 'precip', 'freezing', 'phenomena','wind dir','wind speed'])

    for index, row in df2.iterrows():
        metar = row["metar"]
        valid = row["valid"]
        #print(metar)
        (ceiling, wind, precip, freezing, phenomena,wdir,wspeed) = bad_weather_classes(row["metar"])
        #comment for wind direction and speed

        if ceiling is not None and wind is not None and precip is not None and freezing is not None and phenomena is not None:
            row_df = pd.DataFrame({'metar': metar, 'valid': valid, 'ceiling': ceiling, 'wind': wind, 'precip': precip, 'freezing': freezing, 'phenomena': phenomena,'wind dir':wdir,'wind speed':wspeed}, index=[0])
            final = pd.concat([final, row_df], ignore_index=True)

    final.to_csv(f"{stations}_{startts:%Y%m%d%H%M}_{endts:%Y%m%d%H%M}_final.csv", index=False)

    final['valid'] = pd.to_datetime(final['valid'])

    return final


In [4]:
#stack_df is the needed df which has timestamps 15 mins interval
#day_df is the data we have
#stack id  is id
def stack_count(stack_df,day_df,stack_id):
    # day_stack_dfis is the data we have for specific stack of a day
    day_stack_df = day_df[day_df[stack_id] == 1]
    

    for index, row in stack_df.iterrows():
        specific_timestamp=row['TimeStamp']
        
        filtered_day_15min = day_stack_df[day_stack_df['Time_rounded'] == specific_timestamp]
        
        id_sum_value = filtered_day_15min[stack_id].sum()
        stack_df.at[index, stack_id]=id_sum_value
        
        id_holding_mean = filtered_day_15min['Holding Time (minutes)'].mean(numeric_only=True)
        stack_df.at[index, f'Holding Time {stack_id}']=id_holding_mean 

        id_max_value = filtered_day_15min['Holding Time (minutes)'].max()
        stack_df.at[index, f'{stack_id} Max']=id_max_value

        id_min_value = filtered_day_15min['Holding Time (minutes)'].min()
        stack_df.at[index, f'{stack_id} Min']=id_min_value
        
        
        
    return stack_df

In [5]:
#stack_df is the needed df which has timestamps 15 mins interval
#day_df is the data we have
#stack id  is id
def stack_no_fix_def(day_df,stack_id):
    # day_stack_dfis is the data we have for specific stack of a day
    stack_no_fix = day_df[day_df[stack_id] == 0]
    return stack_no_fix

In [6]:
#stack_df is the needed df which has timestamps 15 mins interval
#day_df is the data we have
#stack id  is id
def stacks_count(stack_df,day_df):
    for index, row in stack_df.iterrows():
        specific_timestamp=row['TimeStamp']
        
        filtered_day_15min = day_df[day_df['Time_rounded'] == specific_timestamp]
        
        WTC_L = filtered_day_15min['WTC L'].sum()
        stack_df.at[index, 'WTC L']=WTC_L
        
        WTC_M = filtered_day_15min['WTC M'].sum()
        stack_df.at[index, 'WTC M']=WTC_M
        
        WTC_H = filtered_day_15min['WTC H'].sum()
        stack_df.at[index, 'WTC H']=WTC_H
        
        WTC_J = filtered_day_15min['WTC J'].sum()
        stack_df.at[index, 'WTC J']=WTC_J
        
        Engine_Jet = filtered_day_15min['Engine Jet'].sum()
        stack_df.at[index, 'Engine Jet']=Engine_Jet
        
        Engine_Turboprop = filtered_day_15min['Engine Turboprop/shaft'].sum()
        stack_df.at[index, 'Engine Turboprop/shaft']=Engine_Turboprop

        Runway09L = filtered_day_15min['Runway 09L'].sum()
        stack_df.at[index, 'Runway 09L']=Runway09L

        Runway09R = filtered_day_15min['Runway 09R'].sum()
        stack_df.at[index, 'Runway 09R']=Runway09R

        Runway27L = filtered_day_15min['Runway 27L'].sum()
        stack_df.at[index, 'Runway 27L']=Runway27L
        
        Runway27R = filtered_day_15min['Runway 27R'].sum()
        stack_df.at[index, 'Runway 27R']=Runway27R
       
       
    
    return stack_df

In [7]:
def calculate_acute_angle(angle1, angle2):

    # Calculate the absolute difference between the angles
    angle_difference = abs(angle1 - angle2)

    # Take the minimum of the absolute difference and its complement
    acute_angle = min(angle_difference, 360 - angle_difference)

    return acute_angle

In [8]:
def cal_crosswind(stack_df):
    
    runway_columns_value = stack_df.filter(regex='^Runway').copy()
    runway_columns = stack_df.filter(regex='^Runway').copy()
    
    # Find the maximum value in each row
    runway_columns_value['Max_Value'] = runway_columns_value.max(axis=1)

    runway_columns['Max_Column'] = runway_columns.idxmax(axis=1)

    
    for index, row in runway_columns_value.iterrows():
        if row['Max_Value']==0:
            stack_df.at[index, 'Crosswind Component']=0
            stack_df.at[index, 'Headwind Component']=0
        else:
            wind_dir=stack_df.at[index,'wind dir']
            wind_speed=stack_df.at[index,'wind speed']

            if ('09' in runway_columns.at[index,'Max_Column']):
                angular_dif= calculate_acute_angle(wind_dir,90)
                
            else:
                angular_dif= calculate_acute_angle(wind_dir,270)

            cross_wind = np.round(np.abs(wind_speed * np.sin(np.radians(angular_dif))), 0)
            head_wind = np.round(np.abs(wind_speed * np.cos(np.radians(angular_dif))), 0)
            
            stack_df.at[index,'Crosswind Component']=cross_wind
            stack_df.at[index,'Headwind Component']=head_wind
    return stack_df
        

        

In [9]:
def No_of_landings (stack_df,day_df):
    no_of_landings=0
    for index, row in stack_df.iterrows():
        specific_timestamp=row['TimeStamp']

        filtered_day_15min = day_df[day_df['Time_rounded_last_ts'] == specific_timestamp]

        no_of_landings_1 = filtered_day_15min['Time_rounded_last_ts'].count()
        no_of_landings_2 = (filtered_day_15min['Time_rounded_last_ts']-pd.to_timedelta(15, unit='m')).count()
        no_of_landings_3 = (filtered_day_15min['Time_rounded_last_ts']-pd.to_timedelta(30, unit='m')).count()
        no_of_landings_4 = (filtered_day_15min['Time_rounded_last_ts']-pd.to_timedelta(45, unit='m')).count()
        no_of_landings=no_of_landings_1+no_of_landings_2+no_of_landings_3+no_of_landings_4
        stack_df.at[index, 'No of Landings 1HR']=no_of_landings
    return stack_df

In [10]:
def stack_no_fix_count(stack_no_fix_day, stack_df):
    for index,row in stack_df.iterrows():
        specific_timestamp=row['TimeStamp']
        filtered_stack_15min = stack_no_fix_day[stack_no_fix_day['Time_rounded_last_ts'] == specific_timestamp]
        #print(filtered_stack_15min)
        if filtered_stack_15min.empty:
            stack_df.at[index, 'No stack']=0
        else:
            no_stack_total=0
            no_stack_total = filtered_stack_15min['Time_rounded_last_ts'].count()
            
            #time.sleep(60) 
            stack_df.at[index, 'No stack']=no_stack_total

    return stack_df

In [11]:
original_df=pd.read_csv('EGLL_2023-04-01_00-00_till_2023-08-31_23-30.csv')
delay_index_df=pd.read_csv('Delayindex_of_EGLL_from_2023-03-25_01_00_till_2024-03-23_23_30')
station=['EGLL']


original_df = original_df.sort_values(by='TimeStamp')
original_df['Time leaving fix'] = pd.to_datetime(original_df['Time leaving fix'])
original_df['Last Timestamp'] = pd.to_datetime(original_df['Last Timestamp'])


delay_index_df=delay_index_df.fillna(0)
delay_index_df=delay_index_df.drop(columns=["Unnamed: 0",'airportIcao','from_utc','departures_numTotal','departures_numCancelled','departures_medianDelay','arrivals_numTotal','arrivals_medianDelay'])
delay_index_df['to_utc'] = pd.to_datetime(delay_index_df['to_utc'] )
delay_index_df['to_utc']=delay_index_df['to_utc'].dt.tz_localize(None)
delay_index_df = delay_index_df.sort_values(by='to_utc')

# Round the timestamps to the nearest 15-minute interval
original_df['Time_rounded'] = original_df['Time leaving fix'].dt.ceil(freq='15min')
original_df['Time_rounded_last_ts'] = original_df['Last Timestamp'].dt.ceil(freq='15min')
#print(original_df)

first_date=(original_df['TimeStamp']).iloc[0]

# Convert the 'date' column to datetime format
first_date = pd.to_datetime(first_date)
first_date = first_date.date()

last_date=(original_df['TimeStamp']).iloc[-1]
# Convert the 'date' column to datetime format
last_date = pd.to_datetime(last_date)
last_date = last_date.date()

In [12]:
range_ap=pd.date_range(first_date,last_date)

weather=main_weather(first_date-pd.Timedelta(days=1),last_date,station)
weather_sorted = weather.sort_values(by='valid')



stack_df = pd.DataFrame(columns=['Holding Time Big','Holding Time Ock','Holding Time Bov','Holding Time Lam','TimeStamp','Big', 'Ock', 'Bov','Lam'])
stack_ids=['Big', 'Ock', 'Bov','Lam']



for i in range_ap:
    date_to_filter=i.date()
    day=date_to_filter.day
    month=date_to_filter.month

    day_df = original_df.loc[(original_df['day'] == day) & (original_df['month'] == month)]

    stack_no_fix_day = day_df.copy()
    
    
    if not day_df.empty :

        stack_day_df = pd.DataFrame({'TimeStamp': pd.date_range(start=(i+ pd.to_timedelta(15, unit='m')), periods=96, freq='15min')})

        
        current_time=stack_day_df['TimeStamp']
        stack_day_df[f'Decimal Hours'] =(current_time.dt.hour + current_time.dt.minute / 60).round(2)
        
        stack_day_df['day'] = stack_day_df['TimeStamp'].dt.day
        stack_day_df['month'] = stack_day_df['TimeStamp'].dt.month
        stack_day_df['year'] = stack_day_df['TimeStamp'].dt.year
        stack_day_df['day_of_week'] = stack_day_df['TimeStamp'].dt.weekday # Monday=0, Sunday=6

        for index, row in stack_day_df.iterrows():
            for dow in [0,1,2,3,4,5,6]:
                if stack_day_df.at[index,'day_of_week'] == dow:
                    stack_day_df.at[index,f'Day_{dow}']=1
                else:
                    stack_day_df.at[index,f'Day_{dow}']=0
        

        
        for stack_id in stack_ids:
            stack_day_df=stack_count(stack_day_df,day_df,stack_id)
            stack_day_df.fillna(0, inplace=True)
            stack_day_df[f'Holding Time {stack_id}'] = (stack_day_df[f'Holding Time {stack_id}'] * 2).round() / 2
            
        stack_day_df=stacks_count(stack_day_df,day_df)
        stack_day_df=No_of_landings (stack_day_df,day_df)
        
        for stack_id in stack_ids:
            stack_no_fix_day=stack_no_fix_def(stack_no_fix_day,stack_id)

        stack_day_df= stack_no_fix_count(stack_no_fix_day, stack_day_df)
        
        print(f"Data for {date_to_filter} Processed.")

        # Concatenate the DataFrames
        #stack_df increases by every loop
        stack_df = pd.concat([stack_df, stack_day_df])
        #stack_df.to_csv('stack_temp_1')
    else:
        print(f"No data for {date_to_filter}. Skipping to next date.")
        continue
        
merged_weather = pd.merge_asof(stack_df, weather_sorted, left_on='TimeStamp', right_on='valid', direction='backward')
stack_df=merged_weather
stack_df=stack_df.drop(columns=['metar','valid'])
stack_df=cal_crosswind(stack_df)

merged_delay_index = pd.merge_asof(stack_df, delay_index_df, left_on='TimeStamp', right_on='to_utc', direction='backward')
stack_df=merged_delay_index
stack_df=stack_df.drop(columns=['to_utc'])

#print(stack_df)


Downloading: EGLL


C:\Users\mic__\AppData\Local\Temp\ipykernel_37348\4032102984.py:86: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final = pd.concat([final, row_df], ignore_index=True)
C:\Users\mic__\.conda\envs\traffic\lib\site-packages\metar\Metar.py:502: RuntimeWarning: Unparsed groups in body 'PROB30' while processing 'COR EGLL 272020Z AUTO 11010KT 8000 -RA BKN008/// OVC024/// //////TCU 10/09 Q1009 RERA PROB30 TEMPO 4000 RADZ BKN004'
  warnings.warn(message, RuntimeWarning)
C:\Users\mic__\.conda\envs\traffic\lib\site-packages\metar\Metar.py:502: RuntimeWarning: Unparsed groups in body '13/NOSIG06' while processing 'COR EGLL 262120Z AUTO 08008KT 9999 NCD 13/NOSIG06 Q1031'
  warnings.warn(message, RuntimeWarning)
C:\Users\mic__\.conda\envs\traffic\lib\sit

Data for 2023-04-01 Processed.


C:\Users\mic__\AppData\Local\Temp\ipykernel_37348\2365741133.py:62: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  stack_df = pd.concat([stack_df, stack_day_df])


Data for 2023-04-02 Processed.
Data for 2023-04-03 Processed.
Data for 2023-04-04 Processed.
Data for 2023-04-05 Processed.
Data for 2023-04-06 Processed.
Data for 2023-04-07 Processed.
Data for 2023-04-08 Processed.
Data for 2023-04-09 Processed.
Data for 2023-04-10 Processed.
Data for 2023-04-11 Processed.
Data for 2023-04-12 Processed.
Data for 2023-04-13 Processed.
Data for 2023-04-14 Processed.
Data for 2023-04-15 Processed.
Data for 2023-04-16 Processed.
Data for 2023-04-17 Processed.
Data for 2023-04-18 Processed.
Data for 2023-04-19 Processed.
Data for 2023-04-20 Processed.
Data for 2023-04-21 Processed.
Data for 2023-04-22 Processed.
Data for 2023-04-23 Processed.
Data for 2023-04-24 Processed.
Data for 2023-04-25 Processed.
Data for 2023-04-26 Processed.
Data for 2023-04-27 Processed.
Data for 2023-04-28 Processed.
Data for 2023-04-29 Processed.
Data for 2023-04-30 Processed.
Data for 2023-05-01 Processed.
Data for 2023-05-02 Processed.
Data for 2023-05-03 Processed.
Data for

In [13]:
first_date = first_date.strftime('%Y-%m-%d %H:%M:%S')
last_date = last_date.strftime('%Y-%m-%d %H:%M:%S')
first_date=first_date.replace(" ", "_").replace(":", "-").replace('-00+0000','')
last_date=last_date.replace(" ", "_").replace(":", "-").replace('-00+0000','')
stack_df.to_csv(f"{station}_{first_date}_till_{last_date}_stack.csv", index=True)